In [2]:
import dualcodec
import torchaudio
from IPython.display import Audio

/home/vansh/dualcodec-vansh/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_id = "12hz_v2"
my_model_path = "/home/vansh/dualcodec-vansh/averaged_models/"
model_name = "averaged_model_step_0260000_decay_0.85.safetensors"

my_model = dualcodec.get_model(base_id, my_model_path, name=model_name, strict=True)
my_inference = dualcodec.InferenceWhisper(dualcodec_model=my_model)

/home/vansh/dualcodec-vansh/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


pretrained_model_path /home/vansh/dualcodec-vansh/averaged_models
Loading model from here /home/vansh/dualcodec-vansh/averaged_models/averaged_model_step_0260000_decay_0.85.safetensors
Model loaded


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 107153.75it/s]


In [4]:
my_inference.semantic_cfg.semantic_model.encoder.causal_mask[0][0][1][:10]

tensor([0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf], device='cuda:0')

In [5]:
audio, sr = torchaudio.load("audio_samples/2400.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()
print("*")
print(f"audio.shape: {audio.shape}")
Audio(audio.squeeze(0), rate=24000)

*
audio.shape: torch.Size([1, 1, 479036])


In [6]:
semantic_codes, acoustic_codes = my_inference.encode(audio, n_quantizers=8)

`sdpa` attention does not support `output_attentions=True` or `head_mask`. Please set your attention to `eager` if you want any of these features.


audio.shape in encode: torch.Size([1, 1, 479036])
features.shape after: torch.Size([1, 512, 997])


In [7]:
print(semantic_codes.shape)
print(acoustic_codes.shape)

torch.Size([1, 1, 249])
torch.Size([1, 7, 249])


In [8]:
out_audio = my_inference.decode(semantic_codes, acoustic_codes)
print("*")
Audio(out_audio.cpu().squeeze(0), rate=24000)

*
